In this notebook, we will see a practical implementation of a block encoding technique based on linear combination of unitaries (LCU), which can be useful to simulate dynamics of quantum system. 

## Linear combination of unitaries
Even if quantum circuit are described by unitary matrices, quantum computers are still able to perform non unitary operations, by using higher dimensional space. In practice, any matrix $H$ can be block encoded into a unitary matrix of higher dimension as

\begin{equation}
V=\begin{pmatrix}H&*\\*&* \end{pmatrix},
\end{equation}

where $*$ denote arbitrary numbers such that $V$ is unitary.

The key ingredient is to write $H$ as a linear combination of $K$ unitaries ([LCU](https://arxiv.org/abs/1202.5822)):

\begin{equation}
H = \sum_{k=0}^{K-1} \alpha_k U_k,
\end{equation}

with $\alpha_k \in \mathbb{C}$\ $\{0\}$ and $U_k$ unitary. By assuming that $H$ is hermitian, whih is the case for the Hamiltonian of any physical system, we can find a simple decomposition by projecting it onto the Pauli basis. We note that, in  general, the performance is driven by the 1 norm of the decomposition, and thus it remains a non trivial task to choose a suitable one.

## Block encoding
Block encoding relies on two important subroutines: 

 - The PREPARE subroutine encodes the coefficients of the Hamiltonian in the amplitudes of the quantum state as
 
\begin{equation}
\text{PREPARE}|\bar{0}\rangle = \sum_{k=0}^{K-1} \sqrt{\frac{\alpha_k}{\|\vec{\alpha}\|_1}} |k\rangle,
\end{equation}

where $|\bar{0}\rangle = |0^{\otimes \lceil \log_2{K}\rceil} \rangle$ is the ancillary register. Note that we can always assume that $\alpha_k\in \mathbb{R}^+$ by assimilating the phase into the corresponding unitary.


This can be achieved using the [`qml.MottonenStatePreparation`](https://docs.pennylane.ai/en/stable/code/api/pennylane.MottonenStatePreparation.html) function with the vector $\vec{\alpha} = (\alpha_1, \cdots, \alpha_n)$. 

 - The SELECT subroutine instead applies the $k$-th unitary $U_k$ on $|\psi\rangle$, when given access to the state $|k\rangle$ as follows
 
\begin{equation}
\text{SELECT} |k\rangle |\psi\rangle  = |k\rangle U_k|\psi \rangle.
\end{equation}

This can be acchieved using control [`qml.ctrl`](https://docs.pennylane.ai/en/stable/code/api/pennylane.ctrl.html) operations on the ancila qubits. 

- $H$ can then be block encoded using the following operation: $\|\vec{\alpha}\|_1 \cdot$ PREPARE$^\dagger$ SELECT PREPARE $|\bar{0}\rangle$.





## Apply the LCU to a quantum state
Let's focus on the particular example where the LCU is composed of $K=4$ terms, and you want to apply $H$ to a quantum state $|\psi\rangle$. We can show that
\begin{equation}
\text{PREPARE}^\dagger \text{ SELECT PREPARE} |\bar{0}\rangle |\psi\rangle = \frac{1}{\|\vec{\alpha}\|_1}|\bar{0}\rangle \sum_{k=0}^{K-1} \alpha_k U_k|\psi \rangle + |\Phi\rangle^\perp,
\end{equation}
where $|\Phi\rangle^\perp$ is some orthogonal state obtained when the algorithm fails. The desired state, up to the normalisation factor, can then be obtained via post selecting on $|\bar{0}\rangle$. The following circuit summaries the result.
<div>
<img src="LCU/LCU.png" width="600"/>
</div>


Let start by setting the problem. We choose a random Hermitian matrix of size $2^n \times 2^n$, with $n=2$.

In [1]:
import numpy as np
import pennylane as qml

n = 2 #physical system size 

shape = (2**n,2**n) 
H     = np.random.uniform(-1, 1, shape) + 1.j * np.random.uniform(-1, 1, shape)  # random matrix
H     = H + H.conjugate().transpose()  # makes it hermitian


LCU    = qml.pauli_decompose(H)  #Projecting the Hamiltonian onto the Pauli basis 
alphas = LCU.terms()[0] 

print('LCU decomposition: \n',LCU)

phases = np.angle(alphas)
coeffs = np.abs(alphas)

coeffs  = np.sqrt(coeffs)
coeffs /= np.linalg.norm(coeffs, ord=2)  # normalise the coefficients

unitaries = [qml.matrix(op) for op in LCU.terms()[1] ]


K = len(coeffs) # number of terms in the decomposition 
a = int(np.ceil(np.log2(K))) # number of ancilla qubits 



LCU decomposition: 
   (-1.5375340914305804) [I0 I1]
+ (-0.9452950224630736) [X0 X1]
+ (-0.6296993210443482) [Y0 Y1]
+ (-0.32063737205210163) [X0 Z1]
+ (-0.31505437298074723) [Z0 Y1]
+ (-0.21116532369574514) [I0 X1]
+ (-0.16714633455719163) [I0 Y1]
+ (-0.1390394470702624) [Z0 I1]
+ (-0.10118339947012045) [Y0 Z1]
+ (-0.07884291789054043) [Z0 Z1]
+ (-0.015069956621447167) [X0 I1]
+ (-0.0033594700837916713) [Y0 X1]
+ (0.013391971618752385) [I0 Z1]
+ (0.04578094027986135) [Y0 I1]
+ (0.3390802083627884) [Z0 X1]
+ (0.9278195429979078) [X0 Y1]


In [2]:
wires_ancilla = np.arange(a)
wires_physical = np.arange(a,a+n)

def Block_encoding(coeffs, phases,  unitaries):
    
    # Prep
    qml.MottonenStatePreparation(coeffs , wires = wires_ancilla)
    
    # Select
    for k in range(K):
        qml.ctrl(qml.QubitUnitary, control = wires_ancilla, 
                 control_values = np.binary_repr(k,width=a))(
                np.exp(1.j*phases[k])*unitaries[k],wires = wires_physical)
  
    # Reverse Prep
    qml.adjoint(qml.MottonenStatePreparation)(coeffs, wires = wires_ancilla)
        
matrix = qml.matrix(Block_encoding)(coeffs, phases, unitaries)
block_matrix = np.linalg.norm(alphas,ord=1)*matrix[:2**n,:2**n]

print("LCU block encoding:\n",np.round(block_matrix,3),'\n')

print("Hamiltonian: \n", np.round(H,3),'\n')

print('Error: ',np.linalg.norm(block_matrix-H),'\n')

print('V is unitary: ', np.allclose(np.eye(matrix.shape[0]),np.dot(matrix.conjugate().transpose(),matrix)))


LCU block encoding:
 [[-1.742+0.j     0.128+0.482j -0.336+0.055j -0.316-0.924j]
 [ 0.128-0.482j -1.611+0.j    -1.575+0.931j  0.306-0.147j]
 [-0.336-0.055j -1.575-0.931j -1.306+0.j    -0.55 -0.148j]
 [-0.316+0.924j  0.306+0.147j -0.55 +0.148j -1.491+0.j   ]] 

Hamiltonian: 
 [[-1.742+0.j     0.128+0.482j -0.336+0.055j -0.316-0.924j]
 [ 0.128-0.482j -1.611+0.j    -1.575+0.931j  0.306-0.147j]
 [-0.336-0.055j -1.575-0.931j -1.306+0.j    -0.55 -0.148j]
 [-0.316+0.924j  0.306+0.147j -0.55 +0.148j -1.491+0.j   ]] 

Error:  1.466247150847548e-15 

V is unitary:  True


/Users/oriel.kiss/Library/Python/3.9/lib/python/site-packages/pennylane/ops/op_math/controlled.py:253: UserWarning: Specifying control values as a string is deprecated. Please use Sequence[Bool]
  warnings.warn(


We see that $H$ is exactly block encoded into a larger unitary matrix, which can be run on a quantum computer.

## Application to quantum simulation
The main problem in quantum chemistry is to be able to extract useful information about quantum state. For instance, the energy of a state can be obtained with the quantum phase estimation algorithm, which relies on performing time evolution. Trotter-Suzuki decompositions are popular techniques to evolve a quantum state, by approximating the time evolution operator using product formulas. For example, the first order product formula for a Hamiltonian of the form $H=\sum_{l=1}^{L} H_l$, where the $H_l$ terms are only *hermitian*, is given by

\begin{equation}
e^{-iHt} \approx \prod_{l=1}^{L}e^{-iH_lt}.
\end{equation}

While these methods already run in polynomial time, better complexity can be achieved by instead expanding the time evolution operator into a [Taylor serie](https://arxiv.org/abs/1412.4687), and truncate it to some order $M$. Hence, we can write 

\begin{equation}
e^{-iHt} = \sum_{k=0}^{\infty} \frac{(-iHt)^k}{k!} \approx \sum_{k=0}^{M} \frac{(-iHt)^k}{k!},
\end{equation}

which can be brought into a LCU form as above, since $H^k$ is hermitian. 

As a concrete example, let us consider a first order truncation

\begin{equation}
e^{-iHt} \approx \mathbb{1} -iHt,
\end{equation}

whose simulation can be recast into the problem we just solved!

## Conclusions

We learnt how to decompose a hermitian matrix into a linear combination of Pauli operators, which can be block encoded into a larger unitary matrix. This scheme is useful to perform time evolution via Taylor expansion, which is a recurring sub routine in quantum algorithms.

## References 
[[Childs12]](https://arxiv.org/abs/1202.5822) Andrew M. Childs, Nathan Wiebe, *Hamiltonian Simulation Using Linear Combinations of Unitary Operations* (2012)

[[Berry14]](https://arxiv.org/abs/1412.4687) Dominic W. Berry, Andrew M. Childs, Richard Cleve, Robin Kothari and Rolando D. Somma *Simulating Hamiltonian dynamics with a truncated Taylor series* (2014)
